# Geoenrichment - Introduction to Enrich

Discovering what is available and distilling this down to something usable is the first step in analysis. Consequently, this is the first piece of functionality we added support for, _introspection_. This provides the ability to discover what countries are available, and within a country, what enrichment variables are available and what travel modes are available for dynamically creating trade areas.

In [16]:
from arcgis.gis import GIS
from arcgis.geoenrichment import get_countries, Country

from demo_data import demo_data

## GIS _Source_

The source GIS being used determines the countries available. Business Analyst can be accessed either locally (ArcGIS Pro with Business Analyst and Data) or through a connection to a Web GIS (ArcGIS Enterprise or ArcGIS Online). In this case, we are connecting to an instance of ArcGIS Online.

In [2]:
gis_agol = GIS(profile='ba')

gis_agol

GIS @ https://bateam.maps.arcgis.com version:10.2

## Discovering Countries

Since the data is organized into countries, this is the first instrospection step, discovering countries.

### Country Source - local

A local `gis` source can be used by passing in an instance of the `GIS` object created using the `'pro'` keyword. As you can see, I have quite a few datasets installed on my machine.

In [3]:
get_countries(GIS('pro'))

,iso2,iso3,name,vintage,country_id,data_source_id
0,CA,CAN,Canada,2021,CAN_ESRI_2021,LOCAL;;CAN_ESRI_2021
1,US,USA,United States,2020,USA_ESRI_2020,LOCAL;;USA_ESRI_2020
2,US,USA,United States,2022,USA_ESRI_2022,LOCAL;;USA_ESRI_2022


### Country Source - Web GIS

Similarly, we can access the countries available on the Web GIS through the `arcpy.gis.GIS` object. Obviously, if this is ArcGIS Online, this is a _lot_ of countries.

In [4]:
get_countries(gis_agol)

,iso2,iso3,name,alt_name,datasets,default_dataset,continent
0,AL,ALB,Albania,ALBANIA,[ALB_MBR_2021],ALB_MBR_2021,Europe
1,DZ,DZA,Algeria,ALGERIA,[DZA_MBR_2021],DZA_MBR_2021,Africa
2,AD,AND,Andorra,ANDORRA,[AND_MBR_2021],AND_MBR_2021,Europe
3,AO,AGO,Angola,ANGOLA,[AGO_MBR_2021],AGO_MBR_2021,Africa
4,AI,AIA,Anguilla,ANGUILLA,[AIA_MBR_2020],AIA_MBR_2020,North America
...,...,...,...,...,...,...,...
172,VE,VEN,Venezuela,"VENEZUELA, BOLIVARIAN REPUBLIC OF",[VEN_MBR_2020],VEN_MBR_2020,South America
173,VN,VNM,Vietnam,VIET NAM,[VNM_MBR_2020],VNM_MBR_2020,Asia
174,VI,VIR,Virgin Islands,UNITED STATES VIRGIN ISLANDS,[VIR_MBR_2020],VIR_MBR_2020,North America
175,ZM,ZMB,Zambia,ZAMBIA,[ZMB_MBR_2021],ZMB_MBR_2021,Africa


## Creating a `Country`

Before digging into enrichment variables, we need to create a `Country` object instance. A `Country` is created using the ISO3 code displayed in the data frame above along with the corresponding `gis` source.

In [5]:
usa = Country('USA', gis=GIS('pro'))

usa

<Country - United States 2022 ('local')>

#### Country - Explicit `year`

If recalling from the introspection previously, three vintages of data are available on my machine for the USA; 2019, 2020, and 2021. If a model was developed against a specific country and data vintage, being able to specifically reference this data vintage is possible using the `year` parameter.

In [7]:
usa2020 = Country('USA', gis=GIS('pro'), year=2020)

usa2020

<Country - United States 2020 ('local')>

## Enrichment Variables

Discovering enrichment variables available is possible through the `Country` object's `enrich_variables` property.

In [22]:
ev = usa.enrich_variables

ev.info()
ev.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27975 entries, 0 to 27974
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               27975 non-null  object
 1   alias              27975 non-null  object
 2   data_collection    27975 non-null  object
 3   enrich_name        27975 non-null  object
 4   enrich_field_name  27975 non-null  object
dtypes: object(5)
memory usage: 1.1+ MB


,name,alias,data_collection,enrich_name,enrich_field_name
10444,MP14143a_B_I,2022 Used Insomnia Prescription Drug : Index,HealthPersonalCare,HealthPersonalCare.MP14143a_B_I,HealthPersonalCare_MP14143a_B_I
25221,RACE3BPO20,2020 Pop 3 Races: BL-PI-OTH,raceandhispanicorigin,raceandhispanicorigin.RACE3BPO20,raceandhispanicorigin_RACE3BPO20
14056,X8030_I,2022 Index: Prescription Drugs,Health,Health.X8030_I,Health_X8030_I
2885,AIF60C10,2010 American Indian Females 60-64,agebyracebysex,agebyracebysex.AIF60C10,agebyracebysex_AIF60C10
18019,X3042FY_X_A,2027 Tools/Equipment-Paint/Paper (Renter) : Av...,HousingHousehold,HousingHousehold.X3042FY_X_A,HousingHousehold_X3042FY_X_A


## Filtering Variables

The usefulness of relevant metadata, especially categorical data, cannot be overstated. Using relevant criteria we can quickly identify variables to use for enrichment.

### Get Current Income Metrics

Since a Pandas DataFrame, finding income indicies for use in analysis is relatively straightforward.

In [10]:
inc_vars = ev[
    (ev.alias.str.lower().str.contains('income'))
    & (ev.name.str.endswith('CY'))
].drop_duplicates('name').reset_index(drop=True)

inc_vars

,name,alias,data_collection,enrich_name,enrich_field_name
0,AVGHINC_CY,2022 Average Household Income,AtRisk,AtRisk.AVGHINC_CY,AtRisk_AVGHINC_CY
1,HINC0_CY,2022 HH Income <$15000,Policy,Policy.HINC0_CY,Policy_HINC0_CY
2,HINC15_CY,2022 HH Income $15000-24999,Policy,Policy.HINC15_CY,Policy_HINC15_CY
3,HINC25_CY,2022 HH Income $25000-34999,Policy,Policy.HINC25_CY,Policy_HINC25_CY
4,HINC35_CY,2022 HH Income $35000-49999,Policy,Policy.HINC35_CY,Policy_HINC35_CY
5,HINC50_CY,2022 HH Income $50000-74999,Policy,Policy.HINC50_CY,Policy_HINC50_CY
6,HINC75_CY,2022 HH Income $75000-99999,Policy,Policy.HINC75_CY,Policy_HINC75_CY
7,HINC100_CY,2022 HH Income $100000-149999,Policy,Policy.HINC100_CY,Policy_HINC100_CY
8,HINC150_CY,2022 HH Income $150000-199999,Policy,Policy.HINC150_CY,Policy_HINC150_CY
9,HINC200_CY,2022 HH Income $200000+,Policy,Policy.HINC200_CY,Policy_HINC200_CY


### Get Current Key Metrics

One of the more common example datasets I use when exploring if an idea is vialbe are the current year key metrics.

In [11]:
kv = ev[
    (ev.name.str.contains('CY'))
    & (ev.data_collection.str.lower().str.contains('key'))
].reset_index(drop=True)

kv

,name,alias,data_collection,enrich_name,enrich_field_name
0,TOTPOP_CY,2022 Total Population,KeyUSFacts,KeyUSFacts.TOTPOP_CY,KeyUSFacts_TOTPOP_CY
1,GQPOP_CY,2022 Group Quarters Population,KeyUSFacts,KeyUSFacts.GQPOP_CY,KeyUSFacts_GQPOP_CY
2,DIVINDX_CY,2022 Diversity Index,KeyUSFacts,KeyUSFacts.DIVINDX_CY,KeyUSFacts_DIVINDX_CY
3,TOTHH_CY,2022 Total Households,KeyUSFacts,KeyUSFacts.TOTHH_CY,KeyUSFacts_TOTHH_CY
4,AVGHHSZ_CY,2022 Average Household Size,KeyUSFacts,KeyUSFacts.AVGHHSZ_CY,KeyUSFacts_AVGHHSZ_CY
5,MEDHINC_CY,2022 Median Household Income,KeyUSFacts,KeyUSFacts.MEDHINC_CY,KeyUSFacts_MEDHINC_CY
6,AVGHINC_CY,2022 Average Household Income,KeyUSFacts,KeyUSFacts.AVGHINC_CY,KeyUSFacts_AVGHINC_CY
7,PCI_CY,2022 Per Capita Income,KeyUSFacts,KeyUSFacts.PCI_CY,KeyUSFacts_PCI_CY
8,TOTHU_CY,2022 Total Housing Units,KeyUSFacts,KeyUSFacts.TOTHU_CY,KeyUSFacts_TOTHU_CY
9,OWNER_CY,2022 Owner Occupied HUs,KeyUSFacts,KeyUSFacts.OWNER_CY,KeyUSFacts_OWNER_CY


## Onto Enrich

From here the next step is enriching data using the retrieved variables. As of the 2.0.1 release of the Python API for ArcGIS you can pass this data frame directly into the `enrich` function.

## Enrich Study Area Polygons

In [17]:
study_area_df = demo_data.pdx_coffee_study_areas_3min.df

study_area_df.info()
study_area_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650 entries, 0 to 649
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   LOCNUM  650 non-null    object  
 1   SHAPE   650 non-null    geometry
dtypes: geometry(1), object(1)
memory usage: 10.3+ KB


,LOCNUM,SHAPE
0,105550909,"{""rings"": [[[-13659712.491841972, 5689050.6789..."
1,105633002,"{""rings"": [[[-13683812.381583042, 5712092.0429..."
2,105759815,"{""rings"": [[[-13655812.509662066, 5706201.2159..."
3,177692910,"{""rings"": [[[-13651962.527299367, 5690511.2941..."
4,180308389,"{""rings"": [[[-13640787.578395057, 5708949.7851..."


In [20]:
study_area_enrich_df = usa.enrich(study_area_df, enrich_variables=kv)

study_area_enrich_df.info()
study_area_enrich_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650 entries, 0 to 649
Data columns (total 43 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   locnum              650 non-null    object  
 1   has_data            650 non-null    int32   
 2   aggregation_method  650 non-null    object  
 3   totpop_cy           650 non-null    float64 
 4   gqpop_cy            650 non-null    float64 
 5   divindx_cy          650 non-null    float64 
 6   tothh_cy            650 non-null    float64 
 7   avghhsz_cy          650 non-null    float64 
 8   medhinc_cy          650 non-null    float64 
 9   avghinc_cy          650 non-null    float64 
 10  pci_cy              650 non-null    float64 
 11  tothu_cy            650 non-null    float64 
 12  owner_cy            650 non-null    float64 
 13  renter_cy           650 non-null    float64 
 14  vacant_cy           650 non-null    float64 
 15  medval_cy           650 non-null    floa

,locnum,has_data,aggregation_method,totpop_cy,gqpop_cy,divindx_cy,tothh_cy,avghhsz_cy,medhinc_cy,avghinc_cy,...,medval_cy_i,avgval_cy_i,popgrwcyfy_i,hhgrwcyfy_i,famgrwcyfy_i,mhigrwcyfy_i,pcigrwcyfy_i,dpopwrk_cy_p,dpopres_cy_p,SHAPE
0,105550909,1,BlockApportionment:US.BlockGroups;PointsLayer:...,6138.0,143.0,41.9,2575.0,2.33,119103.0,176581.0,...,231,191,408,294,275,109,79,73.54,26.46,"{""rings"": [[[-122.70728508062291, 45.427343047..."
1,105633002,1,BlockApportionment:US.BlockGroups;PointsLayer:...,39.0,0.0,61.0,13.0,3.00,111816.0,135177.0,...,163,130,-208,0,0,46,121,99.09,0.91,"{""rings"": [[[-122.92377807415231, 45.572420302..."
2,105759815,1,BlockApportionment:US.BlockGroups;PointsLayer:...,11643.0,1335.0,55.8,7136.0,1.44,66266.0,107837.0,...,222,181,304,248,289,302,134,85.79,14.21,"{""rings"": [[[-122.6722509453407, 45.5353649659..."
3,177692910,1,BlockApportionment:US.BlockGroups;PointsLayer:...,5949.0,62.0,53.5,2612.0,2.25,55293.0,75609.0,...,147,116,28,16,-18,88,118,50.23,49.77,"{""rings"": [[[-122.6376659650056, 45.4365507371..."
4,180308389,1,BlockApportionment:US.BlockGroups;PointsLayer:...,12853.0,677.0,68.0,4924.0,2.47,63848.0,81311.0,...,132,108,-100,-119,-146,136,120,43.26,56.74,"{""rings"": [[[-122.53727969104438, 45.552657456..."


## Enrich Point Locations

In [24]:
pt_df = demo_data.pdx_coffee_locations.df

pt_df.info()
pt_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650 entries, 0 to 649
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   LOCNUM  650 non-null    object  
 1   SHAPE   650 non-null    geometry
dtypes: geometry(1), object(1)
memory usage: 10.3+ KB


,LOCNUM,SHAPE
0,105550909,"{""x"": -13661187.466078103, ""y"": 5686302.261006..."
1,105633002,"{""x"": -13683283.32746541, ""y"": 5709438.5596519..."
2,105759815,"{""x"": -13656422.54659419, ""y"": 5704599.4372188..."
3,177692910,"{""x"": -13651479.293286022, ""y"": 5687492.553844..."
4,180308389,"{""x"": -13640908.50575978, ""y"": 5706009.5641631..."


In [25]:
usa.travel_modes

,name,alias,description,type,impedance,impedance_category,time_attribute_name,distance_attribute_name
0,driving_time,Driving Time,Models the movement of cars and other similar ...,AUTOMOBILE,TravelTime,temporal,TravelTime,Kilometers
1,driving_distance,Driving Distance,Models the movement of cars and other similar ...,AUTOMOBILE,Kilometers,distance,TravelTime,Kilometers
2,trucking_time,Trucking Time,Models basic truck travel by preferring design...,TRUCK,TruckTravelTime,temporal,TruckTravelTime,Kilometers
3,trucking_distance,Trucking Distance,Models basic truck travel by preferring design...,TRUCK,Kilometers,distance,TruckTravelTime,Kilometers
4,walking_time,Walking Time,Follows paths and roads that allow pedestrian ...,WALK,WalkTime,temporal,WalkTime,Kilometers
5,walking_distance,Walking Distance,Follows paths and roads that allow pedestrian ...,WALK,Kilometers,distance,WalkTime,Kilometers
6,rural_driving_time,Rural Driving Time,Models the movement of cars and other similar ...,AUTOMOBILE,TravelTime,temporal,TravelTime,Kilometers
7,rural_driving_distance,Rural Driving Distance,Models the movement of cars and other similar ...,AUTOMOBILE,Kilometers,distance,TravelTime,Kilometers


In [26]:
enrich_pt_df = usa.enrich(pt_df, kv, proximity_type='driving_time', proximity_metric='minutes', proximity_value=3)

enrich_pt_df.info()
enrich_pt_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650 entries, 0 to 649
Data columns (total 47 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   locnum              650 non-null    object  
 1   has_data            650 non-null    int32   
 2   area_type           650 non-null    object  
 3   buffer_units        650 non-null    object  
 4   buffer_unit         650 non-null    object  
 5   buffer_radii        650 non-null    float64 
 6   aggregation_method  650 non-null    object  
 7   totpop_cy           650 non-null    float64 
 8   gqpop_cy            650 non-null    float64 
 9   divindx_cy          650 non-null    float64 
 10  tothh_cy            650 non-null    float64 
 11  avghhsz_cy          650 non-null    float64 
 12  medhinc_cy          650 non-null    float64 
 13  avghinc_cy          650 non-null    float64 
 14  pci_cy              650 non-null    float64 
 15  tothu_cy            650 non-null    floa

,locnum,has_data,area_type,buffer_units,buffer_unit,buffer_radii,aggregation_method,totpop_cy,gqpop_cy,divindx_cy,...,medval_cy_i,avgval_cy_i,popgrwcyfy_i,hhgrwcyfy_i,famgrwcyfy_i,mhigrwcyfy_i,pcigrwcyfy_i,dpopwrk_cy_p,dpopres_cy_p,SHAPE
0,105550909,1,Driving Time,Minutes,Minutes,3.0,BlockApportionment:US.BlockGroups;PointsLayer:...,4132.0,40.0,40.5,...,230,198,444,310,286,117,82,75.67,24.33,"{""x"": -122.72053500019666, ""y"": 45.41001299996..."
1,105633002,1,Driving Time,Minutes,Minutes,3.0,BlockApportionment:US.BlockGroups;PointsLayer:...,223.0,0.0,78.6,...,230,190,-36,-77,-111,0,27,95.98,4.02,"{""x"": -122.91902550031074, ""y"": 45.55573200030..."
2,105759815,1,Driving Time,Minutes,Minutes,3.0,BlockApportionment:US.BlockGroups;PointsLayer:...,13632.0,1818.0,56.0,...,213,174,412,377,346,374,142,86.68,13.32,"{""x"": -122.67773100005218, ""y"": 45.52528499988..."
3,177692910,1,Driving Time,Minutes,Minutes,3.0,BlockApportionment:US.BlockGroups;PointsLayer:...,11053.0,145.0,48.6,...,151,127,120,100,57,159,114,44.02,55.98,"{""x"": -122.63332500012554, ""y"": 45.41751899971..."
4,180308389,1,Driving Time,Minutes,Minutes,3.0,BlockApportionment:US.BlockGroups;PointsLayer:...,11636.0,669.0,67.6,...,133,108,-104,-126,-150,150,121,43.15,56.85,"{""x"": -122.53836600036125, ""y"": 45.53415900023..."
